# Usar el archivo de salida del primer filtrado. 
## Lo continuo en esta segunda parte porque ya no aguantaba el kernel, moria

In [1]:
import pandas as pd

In [2]:
from utils_recuperacion_datos import recuperar_barrios_capital,barrios_capital,geolocator

In [3]:
data = pd.read_csv("datosCabaYGBA.csv", low_memory = False)

# Vamos a recuperar los barrios que estan mal catalogados como 'Capital Federal'

In [4]:
data["place_name"].value_counts().head(5)

Capital Federal    40647
Tigre              29404
Pilar              25204
Belgrano           24689
Nordelta           24060
Name: place_name, dtype: int64

In [5]:
datosCabaIncorrectos = data[data["place_name"] == "Capital Federal"]

In [6]:
datosCorrectos = data[data["place_name"] != "Capital Federal"] 

In [35]:
len(datosCabaIncorrectos)

40647

In [8]:
posiciones_a_corregir = datosCabaIncorrectos["lat-lon"].value_counts()

In [9]:
posiciones_a_corregir = posiciones_a_corregir.index

In [10]:
len(posiciones_a_corregir)

13415

In [ ]:
posiciones_corregidas = recuperar_barrios_capital(posiciones_a_corregir)

In [30]:
def corregir_place_name(latlon):
    if pd.notnull(latlon):
        if latlon in posiciones_corregidas:
            return posiciones_corregidas[latlon]
    return "Capital Federal"

In [32]:
datosCabaIncorrectos["place_name"] = datosCabaIncorrectos.apply(lambda row: corregir_place_name(row["lat-lon"]),axis=1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
datosCabaIncorrectos["place_name"].value_counts()

Capital Federal         8756
 Palermo                3462
 Belgrano               2033
 Villa Urquiza          1915
 Caballito              1879
 Núñez                  1592
 Saavedra               1511
 Balvanera              1347
 Recoleta               1210
 Almagro                1197
 Flores                 1164
 San Nicolás            1140
 Villa Crespo           1064
 Colegiales              900
 Villa Ortúzar           895
 Constitución            869
 Coghlan                 730
 Villa Pueyrredón        657
 Monserrat               629
 San Cristóbal           607
 Villa del Parque        567
 Boedo                   556
 La Boca                 541
 Villa Devoto            523
 Parque Chas             501
 Villa General Mitre     396
 Villa Santa Rita        381
 Retiro                  360
 Parque Chacabuco        287
 Floresta                278
 Chacarita               267
 Parque Avellaneda       253
 Villa Luro              239
 Monte Castro            239
 San Telmo    

# De 40647 registros con barrio incorrecto se recuperaron 31891

In [38]:
datos_corregidos_caba = datosCabaIncorrectos[datosCabaIncorrectos["place_name"] != "Capital Federal"]

In [39]:
len(datos_corregidos_caba)

31891

In [41]:
datos_finales = pd.concat([datosCorrectos,datos_corregidos_caba])

In [45]:
datos_finales.to_csv("datosCabaYGBA.csv",header=True, index=False, encoding='utf-8')